In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 50% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_50, _, y_50 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.5, stratify=y_label_0_1, random_state=42)

In [6]:
X_50.shape

(512870, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt50 = DecisionTreeClassifier()
dt50.fit(X_50, y_50)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [22910   168  2406   842    53  1877   169 85272   208    66]
(22910, 56) (22910,)
Save 0 to baseline_Def1.npy
(168, 56) (168,)
Save 1 to baseline_Def2.npy
(2406, 56) (2406,)
Save 2 to baseline_Def3.npy
(842, 56) (842,)
Save 3 to baseline_Def4.npy
(53, 56) (53,)
Save 4 to baseline_Def5.npy
(1877, 56) (1877,)
Save 5 to baseline_Def6.npy
(169, 56) (169,)
Save 7 to baseline_Def8.npy
(85272, 56) (85272,)
Save 8 to baseline_Def9.npy
(208, 56) (208,)
Save 9 to baseline_Def10.npy
(66, 56) (66,)
Save 10 to baseline_Def11.npy
Execution Time: 0.300035 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [141526    143   1435   1511    187 136792     90  60071     93     65]
(141526, 56) (141526,)
Save 0 to BIM_Def1.npy
(143, 56) (143,)
Save 1 to BIM_Def2.npy
(1435, 56) (1435,)
Save 2 to BIM_Def3.npy
(1511, 56) (1511,)
Save 3 to BIM_Def4.npy
(187, 56) (187,)
Save 4 to BIM_Def5.npy
(136792, 56) (136792,)
Save 5 to BIM_Def6.npy
(90, 56) (90,)
Save 7 to BIM_Def

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [24025   207  2233  1096   119  3122   252 82597   240    80]
(24025, 56) (24025,)
Save 0 to baseline_Def1.npy
(207, 56) (207,)
Save 1 to baseline_Def2.npy
(2233, 56) (2233,)
Save 2 to baseline_Def3.npy
(1096, 56) (1096,)
Save 3 to baseline_Def4.npy
(119, 56) (119,)
Save 4 to baseline_Def5.npy
(3122, 56) (3122,)
Save 5 to baseline_Def6.npy
(252, 56) (252,)
Save 7 to baseline_Def8.npy
(82597, 56) (82597,)
Save 8 to baseline_Def9.npy
(240, 56) (240,)
Save 9 to baseline_Def10.npy
(80, 56) (80,)
Save 10 to baseline_Def11.npy
Execution Time: 2.550188 seconds
(341913,)
[0 2 3 4 5 8 9] [126393    572    809      6 182729  31396      8]
(126393, 56) (126393,)
Save 0 to BIM_Def1.npy
(572, 56) (572,)
Save 2 to BIM_Def3.npy
(809, 56) (809,)
Save 3 to BIM_Def4.npy
(6, 56) (6,)
Save 4 to BIM_Def5.npy
(182729, 56) (182729,)
Save 5 to BIM_Def6.npy
(31396, 56) (31396,)
Save 8 to BIM_Def9.npy
(8, 56) (8,)
Save 9 to BIM_Def10.npy
Execution Time: 3.090506 seconds

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [32204   354  2289  1068   164  3867   339 73373   212   101]
(32204, 56) (32204,)
Save 0 to baseline_Def1.npy
(354, 56) (354,)
Save 1 to baseline_Def2.npy
(2289, 56) (2289,)
Save 2 to baseline_Def3.npy
(1068, 56) (1068,)
Save 3 to baseline_Def4.npy
(164, 56) (164,)
Save 4 to baseline_Def5.npy
(3867, 56) (3867,)
Save 5 to baseline_Def6.npy
(339, 56) (339,)
Save 7 to baseline_Def8.npy
(73373, 56) (73373,)
Save 8 to baseline_Def9.npy
(212, 56) (212,)
Save 9 to baseline_Def10.npy
(101, 56) (101,)
Save 10 to baseline_Def11.npy
Execution Time: 0.176071 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [181348   1602   3998   6179   1250 110587    203   6408  29465    771
    102]
(181348, 56) (181348,)
Save 0 to BIM_Def1.npy
(1602, 56) (1602,)
Save 1 to BIM_Def2.npy
(3998, 56) (3998,)
Save 2 to BIM_Def3.npy
(6179, 56) (6179,)
Save 3 to BIM_Def4.npy
(1250, 56) (1250,)
Save 4 to BIM_Def5.npy
(110587, 56) (110587,)
Save 5 to BIM_Def6.npy
(203, 56) (

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}/UNSW_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB50,22910,168,2406,842,53,1877,0,169,85272,208,66,113971
1,XGB50,141526,143,1435,1511,187,136792,0,90,60071,93,65,341913
2,XGB50,100905,40,1270,1729,83,177761,0,617,59410,20,78,341913
3,XGB50,141526,143,1435,1511,187,136792,0,90,60071,93,65,341913
4,XGB50,74365,634,2390,339,176,243568,729,17703,1646,294,69,341913
5,XGB50,162229,412,4605,2516,270,138795,0,440,32079,375,192,341913
6,XGB50,63521,476,6977,2543,159,5339,0,504,261633,566,195,341913
7,XGB50,248176,89,668,1137,187,17004,0,121,72459,1630,442,341913
8,XGB50,130160,299,1025,1354,142,148774,0,12,60040,61,46,341913
9,XGB50,137412,76,1516,1520,156,138683,0,112,62303,50,85,341913
